In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_calendar=pd.read_csv('/kaggle/input/seattle/calendar.csv')
df_listing=pd.read_csv('/kaggle/input/seattle/listings.csv')

# DATA UNDERSTANDING AND PREPARING

In [ ]:
# UNDERSTANDING df_calendar 
df_calendar.head()

In [ ]:
df_calendar.info()

####  All columns except 'price' have all non-null values
####  'date' column is divided into months and years
####  'available' column is converted to 1 and 0  
####  Remove the '$' symbol and change String to Float from 'price' column

In [ ]:
print("Rows = {} ".format(df_calendar.shape[0]))
print("Columns = {}".format(df_calendar.shape[1]))

In [ ]:
# PREPARING df_calendar

df_calendar['month']=pd.DatetimeIndex(df_calendar['date']).month
df_calendar['year']=pd.DatetimeIndex(df_calendar['date']).year
df_calendar['Month_Year']=pd.to_datetime(df_calendar['date']).dt.to_period('M')

df_calendar['available'].replace({'t':1,'f':0},inplace=True)

df_calendar['price']=df_calendar['price'].map(lambda v: float(v[1:].replace(",","")) if type(v) != float else v)

In [ ]:
#Check del número de filas es el mismo y el numero de columnas ha aumentado en 3 y la columna Price es tipo float
print("Rows = {} ".format(df_calendar.shape[0]))
print("Columns = {}".format(df_calendar.shape[1]))
df_calendar.info()

In [ ]:
# UNDERSTANDING df_listing 
df_listing.head()

####  'square_feet' and 'license' columns have most of the fields null, so we can remove them
####  Columns with descriptions or url doesn`t mind to work with them in thios case
####  Columns where we can do missing values: 'reviews_per_month, beds, bedrooms, bathrooms', since from the rest they can be obtained quite precisely
####  Columns 'space,property_type,host_name,host_is_superhost' can be obtained with Mode since they are Categorical Variables and the missing values are not many
####  Remove the '$' symbol and change String to Float from 'price' column

In [ ]:
df_listing.info()

In [ ]:
# PREPARING df_listing

# 1. Removing the columns 
df_listing.drop(columns=['square_feet','license'],inplace=True)

# numerical missing values columns 
df_listing['beds'].fillna(df_listing['beds'].mean(),inplace=True)
df_listing['bedrooms'].fillna(df_listing['bedrooms'].mean(),inplace=True)
df_listing['bathrooms'].fillna(df_listing['bathrooms'].mean(),inplace=True)

# 2. Categorical missing values columns 
df_listing['space'].fillna(df_listing['space'].mode()[0],inplace=True)
df_listing['property_type'].fillna(df_listing['property_type'].mode()[0],inplace=True)
df_listing['host_name'].fillna(df_listing['host_name'].mode()[0],inplace=True)
df_listing['host_is_superhost'].fillna(df_listing['host_is_superhost'].mode()[0],inplace=True)

# 3. Removing the '$' symbol and change String to Float

df_listing['price']=df_listing['price'].map(lambda v: float(v[1:].replace(",","")) if type(v) != float else v)
df_listing['weekly_price']=df_listing['weekly_price'].map(lambda v: float(v[1:].replace(",","")) if type(v) != float else v)
df_listing['monthly_price']=df_listing['monthly_price'].map(lambda v: float(v[1:].replace(",","")) if type(v) != float else v)
df_listing['security_deposit'] =df_listing['security_deposit'] .map(lambda v: float(v[1:].replace(",","")) if type(v) != float else v)
df_listing['cleaning_fee'] =df_listing['cleaning_fee'] .map(lambda v: float(v[1:].replace(",","")) if type(v) != float else v)
df_listing['extra_people'] =df_listing['extra_people'] .map(lambda v: float(v[1:].replace(",","")) if type(v) != float else v)

In [ ]:
# Check again df_listing cleaned and changed String to Float
print("Rows = {} ".format(df_listing.shape[0]))
print("Columns = {}".format(df_listing.shape[1]))
df_listing.info()

# DATA MODELING AND INSIGHTS

### 1. The busiest month in terms of availbility of listings

#### By Year

In [ ]:
# Groupby by year and agregate by mean of the 'available' column, which takes values between 0 and 1. We can see the year with the highest occupancy.
# Data from only two years doesn't make sense
availability_year=df_calendar.groupby('year')['available'].mean().reset_index().rename(columns={'available':'mean_availability'})
availability_year.head()

#### By Month

In [ ]:
# Groupby by month and agregate by mean of the 'available' column, which takes values between 0 and 1. We can see the month with the highest occupancy
availability_month=df_calendar.groupby('month')['available'].mean().reset_index().rename(columns={'available':'mean_availability'})
availability_month.head()

In [ ]:
x=availability_month['month']
y=availability_month['mean_availability']

In [ ]:
sns.barplot(data=availability_month,x='month',y='mean_availability')

In [ ]:
sns.lineplot(x = "month", y = "mean_availability", data = availability_month).set_title("availability per month")

In [ ]:
availability_month.sort_values(by='mean_availability',ascending=False)

#### The months with the highest availability correspond to the Christmas months. January is the month with the lowest availability

#### By Month_Year

In [ ]:
# Groupby by Month_year and agregate by mean of the 'available' column, which takes values between 0 and 1
availability_monthyear=df_calendar.groupby('Month_Year')['available'].mean().reset_index().rename(columns={'available':'mean_availability'})
availability_monthyear.sort_values(by='mean_availability',ascending=False)

In [ ]:
x=availability_monthyear['Month_Year']
y=availability_monthyear['mean_availability']

In [ ]:
sns.barplot(data=availability_monthyear,x='Month_Year',y='mean_availability')

#### The month and year with the highest availability is January 2017. January of the previous year is the month with the lowest availability.

### 2. The highest mean price per night of listings

In [ ]:
mean_price=df_calendar.groupby('month')['price'].mean().reset_index().rename(columns={'price':'mean_price'})

In [ ]:
x=mean_price['month']
y=mean_price['mean_price']

In [ ]:
sns.barplot(data=mean_price,x='month',y='mean_price').set_title("average price per night and month")

In [ ]:
sns.lineplot(x = "month", y = "mean_price", data = mean_price).set_title("average price per night and month")

In [ ]:
mean_price.sort_values(by='mean_price',ascending=False)

#### The month with the most expensive average price per night corresponds to the summer months (June, July and august)

### 3. The most expensive in terms of 'accomodate'

In [ ]:
accommodates=df_listing.groupby('accommodates')['price'].mean().sort_values(ascending=False).reset_index().rename(columns={'price':'mean_price'})

In [ ]:
x=accommodates['accommodates']
y=accommodates['mean_price']

In [ ]:
sns.barplot(data=accommodates,x='accommodates',y='mean_price').set_title("Average price per accomodates")

In [ ]:
sns.lineplot(x = "accommodates", y = "mean_price", data = accommodates).set_title("average price per accommodates")

In [ ]:
accommodates.sort_values(by='mean_price',ascending=False)

#### Accommodations for 1-7 pax have the price in accordance with the number of pax. But above this, the maximum price is found for 11 pax and the rest is not directly in accordance to the price.

# CONCLUSION

#### The months with the highest availability correspond to the months of Christmas (November and December). January is the month with the lowest availability followed by the summer months (July and August)
#### The highest average price per night, we find it in the summer months (June, July and August), which coincide with those with less availability. It makes sense, since the lower the offer, the higher the price
#### The average price per accommodates, we can see that between 1-7 pax, the relationship is direct. But from there no, being 11 pax the highest average price, that is more high than a 16 pax accomodation